## IBM Coursera Assignment on Segmenting and Clustering Neighborhoods in Toronto

#### By Sidharth Sankar Sabat
#### 7th Aug 2020

In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
import requests
import lxml

In [2]:
from bs4 import BeautifulSoup

#### Part 1. Scraping Wikipedia page data into a dataframe

In [3]:
# Obtaining data from Wikipedia url
wikidata = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
btscrap = BeautifulSoup(wikidata, 'lxml')

In [4]:
PostalCode = []
Borough = []
Neighborhood = []

for row in btscrap.find('table').find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > 0:
        PostalCode.append(cells[0].text)
        Borough.append(cells[1].text)
        Neighborhood.append(cells[2].text.strip())

In [5]:
dict = { 'PostalCode' : PostalCode,
       'Borough' : Borough,
       'Neighborhood': Neighborhood}
canpo = pd.DataFrame.from_dict(dict)
canpo.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [6]:
canpo = canpo.replace('\n','', regex=True) 
canpo.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
# Remove 'Not assigned' Boroughs
canpo = canpo[canpo.Borough != 'Not assigned']
canpo.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
# Copy Borough values to 'Not assigned' Neighborhoods
canpo.replace("Not assigned", np.nan, inplace=True)
canpo['Neighborhood'].fillna(canpo['Borough'], inplace=True)
canpo.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
# Group the datagrame by PostalCode
canpo = canpo.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
canpo.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
canpo.shape

(103, 3)

#### Part 2: Adding Latitude and Longitude to the dataframe

In [11]:
# Reading csv file from the url
url = 'https://cocl.us/Geospatial_data'
coordinates = pd.read_csv(url)
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# Rename column in geodata to merge with canpo dataframe
coordinates.rename(columns={"Postal Code" : "PostalCode"}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
# Merging both coordinate and canpo data fraeme through oute join
coords = pd.merge(canpo, coordinates, on = 'PostalCode', how = 'outer')
coords.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Part 3. Exploring the neighborhood throgh maps using Folium package and Foursquare location data

In [14]:
import folium
from geopy.geocoders import Nominatim

In [15]:
# Geographical coordinates for Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [16]:
# Toronto Map using folium package
torontomap = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(coords['Latitude'], coords['Longitude'], coords['Borough'], coords['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(torontomap)  
    
torontomap

In [45]:
# Define Foursquare credentials and version
CLIENT_ID = 'XXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXX' # your Foursquare Secret
VERSION = '20200807'
print('Foursquare credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Foursquare credentails:
CLIENT_ID: XXXXX
CLIENT_SECRET:XXXX


In [18]:
# Exploring a random neighbour
neighborhood_latitude = coords.loc[9, 'Latitude']
neighborhood_longitude = coords.loc[9, 'Longitude']
neighborhood_name = coords.loc[9, 'Neighborhood']

print('Selected random Neighbour is: {} - Latitude and longitude values of the  selected Neighbour are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Selected random Neighbour is: Birch Cliff, Cliffside West - Latitude and longitude values of the  selected Neighbour are 43.692657000000004, -79.2648481.


In [19]:
# Top 10 venues in the selected Neighbour "Birch Cliff, Cliffside West" within 500 meters radius
LIMIT = 10
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
results_data = requests.get(url).json()
results_data

{'meta': {'code': 200, 'requestId': '5f2d060e86af526945a12bd7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 90,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

In [20]:
from pandas.io.json import json_normalize

In [21]:
def getNeighbourCategoryType(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results_data['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(getNeighbourCategoryType, axis=1)

# clean column names
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Japango,Sushi Restaurant,43.655268,-79.385165
3,Poke Guys,Poke Place,43.654895,-79.385052
4,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
5,Indigo,Bookstore,43.653515,-79.380696
6,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
7,CF Toronto Eaton Centre,Shopping Mall,43.654540,-79.380677
8,Old City Hall,Monument / Landmark,43.652009,-79.381744
9,Textile Museum of Canada,Art Museum,43.654396,-79.386500


In [22]:
#Create a function to repeat the process to all neighborhoods
def getTorontoNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results_data = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_data])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
to_nearby_venues = getTorontoNearbyVenues(names=coords['Neighborhood'],
                                   latitudes=coords['Latitude'],
                                   longitudes=coords['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [24]:
print(to_nearby_venues.shape)
to_nearby_venues.head()

(688, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [25]:
count = to_nearby_venues.groupby('Neighborhood').count()
count = count[['Venue']]
count.rename = {'Total Venues' : 'Venue'}
count.head()

,Venue
Neighborhood,
Agincourt,4
"Alderwood, Long Branch",8
"Bathurst Manor, Wilson Heights, Downsview North",10
Bayview Village,4
"Bedford Park, Lawrence Manor East",10


In [26]:
print('No of unique categories are:  {} '.format(len(to_nearby_venues['Venue Category'].unique())))

No of unique categories are:  179 


In [27]:
# One hot encoding
onehotenc = pd.get_dummies(to_nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe and bring to front
neighbor = to_nearby_venues['Neighborhood']
onehotenc.drop(labels=['Neighborhood'], axis=1,inplace = True)
onehotenc.insert(0, 'Neighborhood', neighbor)

onehotenc.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
onehotenc.shape

(688, 179)

In [29]:
# Group rows by neighborhoodand and  mean of the frequency for each category
toronto_grouped = onehotenc.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
toronto_grouped.shape

(95, 179)

In [31]:
# top 5 venues for each neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1  Latin American Restaurant  0.25
2               Skating Rink  0.25
3                     Lounge  0.25
4         Light Rail Station  0.00


----Alderwood, Long Branch----
         venue  freq
0  Pizza Place  0.25
1          Gym  0.12
2     Pharmacy  0.12
3  Coffee Shop  0.12
4         Pool  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0          Coffee Shop   0.2
1  Fried Chicken Joint   0.1
2                 Bank   0.1
3        Deli / Bodega   0.1
4       Sandwich Place   0.1


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0          Thai Restaurant   0.1
1               Restaurant   0.1
2  Comfort Food Restaurant   0.1
3    

                       venue  freq
0       Fast Food Restaurant   1.0
1                       Lake   0.0
2  Latin American Restaurant   0.0
3         Light Rail Station   0.0
4               Liquor Store   0.0


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
                       venue  freq
0                       Park   0.5
1                 Playground   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4         Light Rail Station   0.0


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
             venue  freq
0    Grocery Store   0.1
1   Discount Store   0.1
2   Sandwich Place   0.1
3  Supplement Shop   0.1
4           Bakery   0.1


----Moore Park, Summerhill East----
                       venue  freq
0                       Park  0.33
1                 Restaurant  0.33
2                      Trail  0.33
3  Latin American Restaurant  0.00
4         Light Rail Station  0.00


----New 

                       venue  freq
0          Convenience Store   0.5
1                       Park   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4         Light Rail Station   0.0




In [32]:
# Put into pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Yoga Studio,Dim Sum Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Pool,Coffee Shop,Sandwich Place,Pharmacy,Gym,Pub,Dance Studio,Distribution Center,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bridal Shop,Sandwich Place,Fried Chicken Joint,Middle Eastern Restaurant,Ice Cream Shop,Deli / Bodega,Restaurant,Bank,Diner
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Dim Sum Restaurant,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Thai Restaurant,Café,Pub,Restaurant,Juice Bar,Indian Restaurant,Sushi Restaurant,Italian Restaurant,Comfort Food Restaurant


In [33]:
# K Means Clustering
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [34]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([2, 0, 0, 0, 0, 2, 0, 0, 2, 0])

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto coords to add latitude/longitude for each neighborhood
toronto_merged = coords
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Yoga Studio,Event Space,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2.0,Bar,Yoga Studio,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2.0,Electronics Store,Restaurant,Intersection,Mexican Restaurant,Bank,Medical Center,Breakfast Spot,Rental Car Location,Donut Shop,Dog Run
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Soccer Field,Korean Restaurant,Dessert Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2.0,Bakery,Athletics & Sports,Caribbean Restaurant,Gas Station,Thai Restaurant,Hakka Restaurant,Bank,Fried Chicken Joint,Dim Sum Restaurant,Diner


In [36]:
toronto_merged.shape

(103, 16)

In [37]:
toronto_merged.dropna(inplace=True)
toronto_merged.shape

(99, 16)

In [38]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Yoga Studio,Event Space,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Bar,Yoga Studio,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Electronics Store,Restaurant,Intersection,Mexican Restaurant,Bank,Medical Center,Breakfast Spot,Rental Car Location,Donut Shop,Dog Run
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Soccer Field,Korean Restaurant,Dessert Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Bakery,Athletics & Sports,Caribbean Restaurant,Gas Station,Thai Restaurant,Hakka Restaurant,Bank,Fried Chicken Joint,Dim Sum Restaurant,Diner


In [39]:
# create cluster map
clusters_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(clusters_map)
       
clusters_map

In [40]:
# Cluster 0 records
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0,Coffee Shop,Soccer Field,Korean Restaurant,Dessert Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
6,Scarborough,0,Hobby Shop,Convenience Store,Coffee Shop,Chinese Restaurant,Discount Store,Department Store,Dim Sum Restaurant,Eastern European Restaurant,Drugstore,Donut Shop
8,Scarborough,0,American Restaurant,Motel,Yoga Studio,Dessert Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
9,Scarborough,0,College Stadium,Skating Rink,Café,General Entertainment,Construction & Landscaping,Convenience Store,Drugstore,Donut Shop,Dog Run,Distribution Center
15,Scarborough,0,Chinese Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Fast Food Restaurant,Bank,Breakfast Spot,Pharmacy,Grocery Store,Diner
19,North York,0,Café,Japanese Restaurant,Chinese Restaurant,Bank,Dim Sum Restaurant,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
22,North York,0,Café,Movie Theater,Coffee Shop,Grocery Store,Indonesian Restaurant,Plaza,Ice Cream Shop,Steakhouse,Ramen Restaurant,Dessert Shop
24,North York,0,Coffee Shop,Pharmacy,Pizza Place,Bank,Yoga Studio,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center
26,North York,0,Japanese Restaurant,Gym,Construction & Landscaping,Clothing Store,Discount Store,Caribbean Restaurant,Sporting Goods Shop,Restaurant,Athletics & Sports,Beer Store
27,North York,0,Japanese Restaurant,Gym,Construction & Landscaping,Clothing Store,Discount Store,Caribbean Restaurant,Sporting Goods Shop,Restaurant,Athletics & Sports,Beer Store


In [41]:
# Cluster 1 records
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1,Park,Playground,Department Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
21,North York,1,Park,Comic Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
23,North York,1,Park,Convenience Store,Dessert Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
25,North York,1,Park,Convenience Store,Food & Drink Shop,Dessert Shop,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
48,Central Toronto,1,Park,Trail,Restaurant,Department Store,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
50,Downtown Toronto,1,Park,Playground,Trail,Deli / Bodega,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
64,Central Toronto,1,Park,Trail,Sushi Restaurant,Jewelry Store,Department Store,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
74,York,1,Park,Pool,Women's Store,Construction & Landscaping,Convenience Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center
98,York,1,Park,Comic Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
100,Etobicoke,1,Park,Sandwich Place,Mobile Phone Shop,Department Store,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner


In [42]:
# Cluster 2 records
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2,Bar,Yoga Studio,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,Scarborough,2,Electronics Store,Restaurant,Intersection,Mexican Restaurant,Bank,Medical Center,Breakfast Spot,Rental Car Location,Donut Shop,Dog Run
4,Scarborough,2,Bakery,Athletics & Sports,Caribbean Restaurant,Gas Station,Thai Restaurant,Hakka Restaurant,Bank,Fried Chicken Joint,Dim Sum Restaurant,Diner
5,Scarborough,2,Women's Store,Playground,Dessert Shop,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
7,Scarborough,2,Bus Line,Bakery,Soccer Field,Intersection,Ice Cream Shop,Metro Station,Bus Station,Discount Store,Eastern European Restaurant,Drugstore
10,Scarborough,2,Indian Restaurant,Pet Store,Vietnamese Restaurant,Light Rail Station,Chinese Restaurant,Yoga Studio,Dessert Shop,Donut Shop,Dog Run,Distribution Center
11,Scarborough,2,Middle Eastern Restaurant,Accessories Store,Shopping Mall,Sandwich Place,Bakery,Breakfast Spot,Auto Garage,Curling Ice,Discount Store,Convenience Store
12,Scarborough,2,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Yoga Studio,Dim Sum Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store
13,Scarborough,2,Italian Restaurant,Gas Station,Convenience Store,Noodle House,Pizza Place,Fried Chicken Joint,Chinese Restaurant,Fast Food Restaurant,Thai Restaurant,Bank
17,North York,2,Golf Course,Dog Run,Fast Food Restaurant,Pool,Mediterranean Restaurant,Yoga Studio,Dessert Shop,Donut Shop,Distribution Center,Discount Store


In [43]:
# Cluster 3 records
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,3,Baseball Field,Yoga Studio,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
97,North York,3,Baseball Field,Food Service,Yoga Studio,Dim Sum Restaurant,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store


In [44]:
# Cluster 4 records
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Fast Food Restaurant,Yoga Studio,Event Space,Eastern European Restaurant,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
